In [1]:
using Distributed
@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [15]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [14]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
#plot(temp.t_grid,results.heat_cap)

# data = [results.en_histogram[i] for i in 1:n_traj]
# plot(data)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done


Testing better mc_cycles

In [15]:
@everywhere function parallelmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    return mc_states
end

@everywhere function paralleltest(var)
    if var == -1
        for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var ==0
        Threads.@threads for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:100
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end


In [17]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time2 = @benchmark paralleltest(0)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

BenchmarkTools.Trial: 1920 samples with 1 evaluation.
 Range (min … max):  876.450 μs … 43.225 ms  ┊ GC (min … max):  0.00% … 96.81%
 Time  (median):       1.602 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.593 ms ±  3.994 ms  ┊ GC (mean ± σ):  15.58% ± 10.19%

  ▇█                                                            
  ██▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▂ ▂
  876 μs          Histogram: frequency by time         32.7 ms <

 Memory estimate: 1.70 MiB, allocs estimate: 17775.

In [18]:
println("no parallel = ",time1," parallelthreads = ",time2," parallelcycle = ",time3)

no parallel = Trial

(1.354 ms) parallelthreads = Trial(392.705 μs) parallelcycle = Trial(876.450 μs)


In [22]:
@everywhere function paralleltest(var)
    if var == -1
        for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var ==0
        Threads.@threads for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:1000
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time2 = @benchmark paralleltest(0)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

println("no parallel = ",time1," parallelthreads = ",time2," parallelcycle = ",time3)

no parallel = Trial(16.274 ms) parallelthreads = Trial(4.456 ms) parallelcycle = Trial(11.424 ms)


In [24]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
@benchmark begin 
    for i in 1:1000
        parallelcycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
    end
end


BenchmarkTools.Trial: 38 samples with 1 evaluation.
 Range (min … max):  118.282 ms … 174.666 ms  ┊ GC (min … max): 0.00% … 7.30%
 Time  (median):     131.653 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   134.916 ms ±  12.574 ms  ┊ GC (mean ± σ):  2.70% ± 3.16%

    ▃ ▃▃ ▃  █  ▃█ ▃    ▃   ▃  ▃    ▃                             
  ▇▁█▁██▇█▁▇█▁▇██▇█▇▇▁▁█▁▇▁█▁▁█▁▁▁▁█▇▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▇ ▁
  118 ms           Histogram: frequency by time          175 ms <

 Memory estimate: 23.01 MiB, allocs estimate: 324162.

In [9]:
@benchmark begin
    ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
end

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done
Total number of moves per MC cycle: 

13

equilibration done

...


[-10.883889797317497, -10.431804745433759, -9.99574737212787, -9.577542992257698, -9.182177879418681, -8.787921033113072, -8.415032597157204, -8.052503609832897, -7.695688582446834, -7.356640070754599, -7.034295159245451, -6.7270310455825335, -6.415890542780838, -6.122637725728883, -5.8341208863918785, -5.551259689315166, -5.280678871729084, -5.005277442232245, -4.730671735025874, -4.426748050972376, -4.162053288087249, -3.8118201697941774, -3.489070624361203, -3.139951779223422, -2.837044815519595, -2.5449006568366954, -2.288194524658741, -2.0877040642525357, -1.883551143956719, -1.6618830580577397, -1.4891663632854393, -1.337553462151169]
done
Total number of moves per MC cycle: 

13

equilibration done


MC loop done.


[-21.76448542527407, -20.864765358539646, -19.99269636891073, -19.15493697281766, -18.357619616682374, -17.575687160125995, -16.833466754240042, -16.104230690610883, -15.391133972223683, -14.718280757024107, -14.07310279391676, -13.44995067204151, -12.831050367633143, -12.247488097722556, -11.67487636182148, -11.106036215574068, -10.563048451410294, -10.02101816405747, -9.470832147413903, -8.893377337680015, -8.32579421426003, -7.638394934995636, -6.987729123817622, -6.243411981760421, -5.634190141705224, -5.075123637753679, -4.564592035963244, -4.165291092318767, -3.7628272502477613, -3.3246711371716122, -2.9789477958494084, -2.650587310617719]
done


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 36.696 s (2.69% GC) to evaluate,
 with a memory estimate of 25.64 GiB, over 183505175 allocations.

In [6]:
import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states

function ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham::Bool = true, save::Bool=true, restart::Bool=false,restartindex::Int64=0)
    #restart isn't compatible with saving the hamiltonian at the moment

    if restart == true
        save_ham = false
    end
    
    if save_ham == false
        if restart == false
            #If we do not save the hamiltonian we still need to store the E, E**2 terms at each cycle
            for i_traj = 1:mc_params.n_traj
                push!(mc_states[i_traj].ham, 0)
                push!(mc_states[i_traj].ham, 0)
            end
        end
    end

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    
    #equilibration cycle
    if restart == false
        #this initialises the max and min energies for histograms
        if save_ham == false
            ebounds = [100. , -100.] #emin,emax
        end
        
        Threads.@threads for i = 1:mc_params.eq_cycles
            @inbounds mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
            #verbose way to save the highest and lowest energies

            if save_ham == false
                for i_traj = 1:mc_params.n_traj
                    if mc_states[i_traj].en_tot < ebounds[1]
                        ebounds[1] = mc_states[i_traj].en_tot
                    end

                    if mc_states[i_traj].en_tot > ebounds[2]
                        ebounds[2] = mc_states[i_traj].en_tot
                    end
                end
            end
            #update stepsizes
            if rem(i, mc_params.n_adjust) == 0
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end
        end
    #re-set counter variables to zero
        for i_traj = 1:mc_params.n_traj
            mc_states[i_traj].count_atom = [0, 0]
            mc_states[i_traj].count_vol = [0, 0]
            mc_states[i_traj].count_rot = [0, 0]
            mc_states[i_traj].count_exc = [0, 0]
        end
        #initialise histogram for non-saving hamiltonian 
        if save_ham == false
            delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds)
        end

        println("equilibration done")
        

        if save == true
            save_states(mc_params,mc_states,0)
        end
    end


    #main MC loop

    if restart == false

        Threads.@threads for i = 1:mc_params.mc_cycles
            if save_ham == false
                ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end

    else #if restarting

        for i = restartindex:mc_params.mc_cycles
            if save_ham == false
                ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end 
    end
    println("MC loop done.")
    #Evaluation
    #average energy
    n_sample = mc_params.mc_cycles / mc_params.mc_sample

    if save_ham == true
        en_avg = [sum(mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj] #floor(mc_cycles/mc_sample)
        en2_avg = [sum(mc_states[i_traj].ham .* mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj]
    else
        en_avg = [mc_states[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]
        en2_avg = [mc_states[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
    end


    results.en_avg = en_avg

    #heat capacity
    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]

    #acceptance statistics
    results.count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
    results.count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]

    println(results.heat_cap)

    #energy histograms
    if save_ham == true
        # T = typeof(mc_states[1].ham[1])
        delta_en= initialise_histograms!(mc_params,mc_states,results)
        updatehistogram!(mc_params,mc_states,results,delta_en)
    
    end
    #     for i_traj in 1:mc_params.n_traj
    #         hist = zeros(results.n_bin)#EnHist(results.n_bin, global_en_min, global_en_max)
    #         for en in mc_states[i_traj].ham
    #             index = floor(Int,(en - global_en_min) / delta_en) + 1
    #             hist[index] += 1
    #         end
    #         push!(results.en_histogram, hist)
    #     end
    # end

    #TO DO
    # volume (NPT ensemble),rot moves ...
    # move boundary condition from config to mc_params?
    # rdfs

    println("done")
    return 
end

ptmc_threaded_run! (generic function with 1 method)

In [11]:
@benchmark begin
    ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
end

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.033315931832741515, 0.032883476690382094, 0.03377214263676734, 0.023545083799733302, 0.016148537499605476, 0.011873652465451827, 0.010949893256928783, 0.011846215802770508, 0.011964140851500115, 0.01135260555271364, 0.010171447585564404, 0.0102159769912188, 0.011936222245349971, 0.012703240698300084, 0.012439317915282076, 0.015528745065002377, 0.014522234150351875, 0.010894829251158249, 0.009873041035726381, 0.009949611589284035, 0.008160798663554389, 0.009707159195862621, 0.008819043100577734, 0.010765999982660479, 0.013529749889175555, 0.01354559659378694, 0.011846259232634866, 0.014147335614978676, 0.015228133924754429, 0.015324400141634805, 0.016657036104586333, 0.00955523318297437]
done
Total number of moves per MC cycle: 13



equilibration done


...

MC loop done.
[-22.529123582032287, -21.100848151559564, -19.601784933361316, -17.944520663140537, -16.6177437877423, -15.61820751498729, -14.734644244450132, -13.646697693820238, -12.652338275600272, -11.668990856574606, -10.851314358825077, -10.031601829530512, -9.331279508767079, -8.156095628995239, -7.297243576767305, -6.502072126288781, -5.9420570151467675, -5.4214401617235035, -5.030426358825759, -4.564326217884239, -4.158349651192735, -3.7729564217052265, -3.4225404107267794, -3.0932824577829345, -2.7859054865239634, -2.5167300491083107, -2.261840418333913, -1.9455576321328554, -1.3255159917031185, -0.8897771921968597, -0.4416729736914624, -0.0901391504675201]
done
Total number of moves per MC cycle: 

13



equilibration done


MC loop done.
[-50.17910462511319, -46.19610967914932, -42.65559971232533, -39.23564200877116, -36.60002142187566, -34.385550466634506, -32.32891660704748, -29.854936991387742, -27.39208845300527, -25.127301926096262, -23.065676402684517, -21.258740501586736, -19.783553864281384, -17.538256222437294, -15.777856623691337, -14.183561208278658, -12.866405720686835, -11.764965158888957, -10.941251395105134, -10.027066426310485, -9.018860087835167, -8.095314664179114, -7.292514141039699, -6.60680087313759, -5.914931476775127, -5.287925013698043, -4.410327217270329, -3.529988221286712, -2.4332409515923934, -1.5665821491608267, -0.7434471712683475, -0.19523537464165713]
done


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 14.574 s (34.51% GC) to evaluate,
 with a memory estimate of 25.64 GiB, over 183505571 allocations.

In [21]:
println(Threads.nthreads(), typeof(Threads.nthreads()))

8Int64

In [22]:
# cp("/home/ghun245/RuNNer-master/Brass_potential/","$(pwd())/Brass_potential1")

"/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts/Brass_potential1"

In [23]:
function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
    potential_directory = mkdir("$(pwd())/pots")
    potential_vector = []
    for i = 1:n_threads
        # n_threads potentials
        cp(pot.dir,"$(potential_directory)/potential$i")
        pot_i = AbstractMLPotential("$(potential_directory)/potential$i",pot.atomtype)
        push!(potential_vector,pot_i)
    end
        
    return potential_vector        
end

function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    parallel_states = []

    pot_vector = init_parallel_RuNNer(pot) #initialise state and potentials

    n_threads = Threads.nthreads()
    sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i_thread = 1:n_threads
        for i_eq = 1:sample_index

            mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)#mc cycle
            for i_traj = 1:mc_params.n_traj#check energy bounds
                if mc_states[i_traj].en_tot < ebounds[1]
                    ebounds[1] = mc_states[i_traj].en_tot
                end

                if mc_states[i_traj].en_tot > ebounds[2]
                    ebounds[2] = mc_states[i_traj].en_tot
                end

            end
            if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end

        end
        states_vec = [MCState(mc_states[idx].temp,mc_states[idx].beta,mc_states[idx].config,pot_vector[idx];max_displ = mc_states[idx].max_displ ) for idx in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

        push!(parallel_states,states_vec) #add to vector of parallel states


    end
    delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

    println("equilibration done")
    
    return a,v,r,delta_en

end

parallel_equilibration (generic function with 1 method)